In [231]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'calles_de_medellin_con_acoso.csv', sep=";")

In [ ]:
def correctFormat(text: str) -> tuple:
    textProcess = text[1:-1]
    partB, partA = textProcess.split(",")
    listReturn = [float(partA),float(partB)]
    return tuple(listReturn)


def average() -> float:
    count = 0
    count2 = 0
    for iteration in df['harassmentRisk']:
        if pd.isnull(iteration):
            continue
        else:
            count2 += 1
            count += iteration
    return count / count2

PROMEDIO = average()
df['harassmentRisk'].fillna(PROMEDIO, inplace=True)
print(PROMEDIO)

In [ ]:
def createGrahp() -> dict:
    uniqueOrigins = pd.unique(df['origin'])
    graph = {}
    for origin in uniqueOrigins:
        correctOrigin = correctFormat(origin)
        graph[correctOrigin] = {}

    for index in df.index:
        originIteration = correctFormat(df['origin'][index])
        destinationIteration = correctFormat(df['destination'][index])
        if destinationIteration in graph:
            if df['oneway'][index]:
                graph[originIteration] = graph[originIteration] | {destinationIteration:(df['length'][index],df['harassmentRisk'][index])}
                graph[destinationIteration] = graph[destinationIteration] | {originIteration:(df['length'][index],df['harassmentRisk'][index])}
            else: graph[originIteration] = graph[originIteration] | {destinationIteration:(df['length'][index],df['harassmentRisk'][index])}
        else:
            if df['oneway'][index]:
                graph[destinationIteration] = {}
                graph[originIteration] = graph[originIteration] | {destinationIteration:(df['length'][index],df['harassmentRisk'][index])}
                graph[destinationIteration] = graph[destinationIteration] | {originIteration:(df['length'][index],df['harassmentRisk'][index])}
            else: graph[originIteration] = graph[originIteration] | {destinationIteration:(df['length'][index],df['harassmentRisk'][index])}

    return graph

graphCM = createGrahp()
print(graphCM[(6.2063927, -75.5686884)])

In [ ]:
def DjikstraFirst(graph: dict, origin: tuple, destination: tuple) -> list:
    shortesdDistance = {}
    previusly = {}
    noVisiteds = graph
    CONTROLER = 9999
    shortestPath = []

    for node in noVisiteds:
        shortesdDistance[node] = CONTROLER
    shortesdDistance[origin] = 0

    while noVisiteds:
        minDistance = None
        for node in noVisiteds:
            if minDistance is None:
                minDistance = node
            elif shortesdDistance[node] < shortesdDistance[minDistance]:
                minDistance = node

        pahtOptions = graph[minDistance].items()

        for childNode, weight in pahtOptions:
            if weight[0] + shortesdDistance[minDistance] < shortesdDistance[childNode]:
                shortesdDistance[childNode] = weight[0] + shortesdDistance[minDistance]
                previusly[childNode] = minDistance

        noVisiteds.pop(minDistance)

    actualyNode = destination

    while actualyNode != origin:
        try:
            shortestPath.insert(0, actualyNode)
            actualyNode = previusly[actualyNode]
        except KeyError:
            print("El camino no es posible de encontrar")
            break

    if shortesdDistance[destination] != CONTROLER:
        print("La distancia más corta es de: ", str(shortesdDistance))
        print("El camino más corto está dado por: ", str(shortestPath))

DjikstraFirst(graphCM,(6.2734442, -75.5443961),(6.2094357, -75.5674348))